**Parte 4: Selección**

Vamos a calcular estadísticos de selección en una población de lobos africanos que viven en condiciones extremas (desierto del Sáhara). Para ello vamos a correr selscan, un paquete de estadísticos que funciona con archivos vcf y un "mapa" de sitios. 

El mapa de sitios se hace muy fácilmente: 

In [ ]:
zcat $in/chr38.agw.soft.recode.vcf.recode.vcf.gz | grep -v "^#" | awk '{print $1"\t"$3"\t"$2"\t"$2 }' > $out/chr38.map


Por si acaso, ya tenéis una copia en $in. Vamos a correr nsl:

In [ ]:
selscan=/home/bio/programas/selscan/bin/linux/selscan-2.0.3

$selscan --nsl --vcf $in/chr38.agw.soft.recode.vcf.recode.vcf.gz --map $in/chr38.map --threads 10 --out $out/chr38.agw 

Este comando calcula nSL, un estadístico de selección, en el archivo agw. Para eso usamos 10 hilos (-threads 10). Como podéis ver en la ejecución del programa, por defecto selscan va a eliminar variantes con alelos en frecuencias alélicas menores a 5% (es decir, menores de 5% o mayores de 95%) (--skip-low-freq set. Removing all variants < 0.05.)

Podeis observar el output de nSL con un head. Aunque selscan no genera un header para este archivo, las columnas son:
locusID	physicalPos	'1'_freq	sl1	sl0	unstandardized_nSL

locusID: identificación del locus (no tenemos ninguno en este VCF)
physicalPos: posición en el cromosoma
'1'_freq: frecuencia del alelo '1' (alelo alternativo)
sl1: valor del estadístico sl (único de nSL) en el alelo '1'
sl0: valor del estadístico sl en el alelo '0' (alelo de referencia)
unstandardized_nSL: valor de nSL

In [ ]:
head $out/chr38.agw.nsl.out

Como veis, tenemos un valor de nSL por cada sitio del cromosoma. Pero, ¿cuáles son los relevantes? Para eso tenemos que correr norm, otro programa de selscan que normaliza los cromosomas. 

In [ ]:
norm=/home/bio/programas/selscan/bin/linux/norm
$norm --nsl --files $out/chr38.agw.nsl.out --bins 20

Esto crea el archivo $out/chr38.agw.nsl.out.20bins.norm. Ahora teneis dos columnas mas: el valor de nSL normalizado y una columna final que dice si es significativo o no. Para ver que SNPs están bajo selección, elegid ls que en la última columna tienen un 1:

In [ ]:
cat chr38.agw.nsl.out.20bins.norm | awk -v OFS='\t' '{if ($8 == 1) print $0}' | head

**Pregunta 1**: ¿Qué sitios están bajo selección?

Vamos a plotear esta estadística. Ejecutad:

In [ ]:
python /home/bio/plotting/scatter_plot_script.py $out/chr38.agw.nsl.out.20bins.norm $out/nsl_plot_output.pdf


Descargad el archivo con (scp user@172.64.16.4:/ruta/hacia/tus/archivos/nsl_plot_output.pdf ./) y observadlo. **Pregunta 2**: ¿Qué veis en el plot?
